# 코사인 유사도(Cosine Similarity)
- 코사인 유사도는 두 벡터 간의 코사인 각도를 이용하여 유사도를 구한다.
- 두 벡터 방향이 동일한 경우 - "1"
- 두 벡터 방향이 90도인 경우 - "0"
- 두 벡터 방향이 반대인 경우 - "-1"
 ![코사인 유사도](https://github.com/tenjumh/GraduateSchool/blob/master/Study/NLP_Natural%20Language%20Processing/image/%EC%BD%94%EC%82%AC%EC%9D%B8%EC%9C%A0%EC%82%AC%EB%8F%84.png?raw=True)

- 두 벡터 A, B에 대한 코사인 유사도 식
  ![코사인 유사도식](https://github.com/tenjumh/GraduateSchool/blob/master/Study/NLP_Natural%20Language%20Processing/image/%EC%BD%94%EC%82%AC%EC%9D%B8%EC%9C%A0%EC%82%AC%EB%8F%84%EC%8B%9D.PNG?raw=True)

- 문서 단어 행렬이나 TF-IDF행렬을 통해서 문서의 유사도를 구하는 경우, 문서 단어 행렬이나 TF-IDF행렬이 각각의 특징 벡터 A, B가 된다.

 문서1 : 저는 사과 좋아요<br>
 문서2 : 저는 바나나 좋아요<br>
 문서3 : 저는 바나나 좋아요 저는 바나나 좋아요<br>
 <br>
 - 세 문서의 문서 단어 행렬
 
![행렬](https://github.com/tenjumh/GraduateSchool/blob/master/Study/NLP_Natural%20Language%20Processing/image/%EB%AC%B8%EC%84%9C%20%EB%8B%A8%EC%96%B4%20%ED%96%89%EB%A0%AC_%EC%BD%94%EC%82%AC%EC%9D%B8%EC%9C%A0%EC%82%AC%EB%8F%84.png?raw=True)

In [1]:
# numpy를 통한 코사인 유사도
from numpy import dot
from numpy.linalg import norm
import numpy as np

def cos_sim(A,B):
    return dot(A,B)/(norm(A)*norm(B))

In [3]:
doc1 = np.array([0,1,1,1])
doc2 = np.array([1,0,1,1])
doc3 = np.array([2,0,2,2])

In [7]:
print(cos_sim(doc1, doc2))
print(cos_sim(doc1, doc3))
print(cos_sim(doc2, doc3))

0.6666666666666667
0.6666666666666667
1.0000000000000002


문서1과 문서2의 코사인 유사도와 문서1과 문서3의 코사인 유사도가 동일<br>
문서2와 문서3의 코사인 유사도가 1이 나옴<br>
앞서 1은 두 벡터의 방향이 완전히 동일한 경우에 1이 나오며, 코사인 유사도 관점에서는 유사도의 값이 최대임을 의미.<br>
문서3은 문서2에서 단지 모든 단어의 빈도수가 1씩 증가했을 뿐, 이는 한 문서 내의 모든 단어의 빈도수가 똑같이 증가하는 경우에는 기존의 문서와 코사인 유사도의 값이 "1"

# 유사도 추천 시스템 구현

In [9]:
# 다운로드 링크 : https://www.kaggle.com/rounakbanik/the-movies-dataset
# 총 24개의 열을 가진 45,466개의 샘플로 구성된 영화 정보 데이터
# 사용할 데이터는 영화제목(title), 줄거리(overview)
# 좋아하는 영화를 입력하면 해당 영화의 줄거리와 줄거리가 유사한 영화를 추천

import pandas as pd

data = pd.read_csv('C:/Users/fxk/PycharmProjects/tenjumh/Study/NLP_Natural Language Processing/movies_metadata.csv', low_memory=False)
data.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


- TF-IDF는 데이터에 Null값이 들어 있으면 에러 발생

In [10]:
# null값이 있는지 확인
data['overview'].isnull().sum()

954

In [11]:
# overview에서 Null 값을 가진 경우에는 값 제거
data['overview'] = data['overview'].fillna('')

In [12]:
data['overview'].isnull().sum()

0

In [14]:
#TF-IDF 수행
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words = 'english')
tfidf_matrix = tfidf.fit_transform(data['overview'])
# overview에 대해서 tf-idf 수행
print(tfidf_matrix.shape)

(45466, 75827)


45466개의 영화를 표현하기 위해 75827개의 단어 사용

In [15]:
# 코사인 유사도
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [16]:
# 영화의 타이틀과 인덱스를 가진 테이블 (5개만 출력)
indices = pd.Series(data.index, index=data['title']).drop_duplicates()
print(indices.head())

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64


In [17]:
idx = indices['Father of the Bride Part II']
print(idx)

4


In [18]:
def get_recommendations(title, cosine_sim = cosine_sim):
    # 선택한 영화의 타이틀로부터 해당되는 인덱스를 받아옴. 이제 선택한 영화를 가지고 연산
    idx = indices[title]
    
    #모든 영화에 대해서 해당 영화와의 유사도를 구함
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # 유사도에 따라 영화들을 정렬합니다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 영화를 받아옵니다.
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 영화의 인덱스를 받아옵니다.
    movie_indices = [i[0] for i in sim_scores]

    # 가장 유사한 10개의 영화의 제목을 리턴합니다.
    return data['title'].iloc[movie_indices]

In [19]:
get_recommendations('The Dark Knight Rises')

12481                                      The Dark Knight
150                                         Batman Forever
1328                                        Batman Returns
15511                           Batman: Under the Red Hood
585                                                 Batman
21194    Batman Unmasked: The Psychology of the Dark Kn...
9230                    Batman Beyond: Return of the Joker
18035                                     Batman: Year One
19792              Batman: The Dark Knight Returns, Part 1
3095                          Batman: Mask of the Phantasm
Name: title, dtype: object

In [20]:
get_recommendations('Toy Story')

15348                                     Toy Story 3
2997                                      Toy Story 2
10301                          The 40 Year Old Virgin
24523                                       Small Fry
23843                     Andy Hardy's Blonde Trouble
29202                                      Hot Splash
43427                Andy Kaufman Plays Carnegie Hall
38476    Superstar: The Life and Times of Andy Warhol
42721    Andy Peters: Exclamation Mark Question Point
8327                                        The Champ
Name: title, dtype: object

In [21]:
get_recommendations('Hot Splash')

14979                 Blackwoods
0                      Toy Story
176                     Mad Love
11336    The Howards of Virginia
45392          Hopeless Romantic
27155          The Turning Point
14001          Randy Rides Alone
6713                 Out of Time
16740       Take Me Home Tonight
467             I'll Do Anything
Name: title, dtype: object